# **Import Required Libraries**

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
from keras.models import load_model
from tensorflow.keras.models import model_from_json

# **Load Saved Model**

In [ ]:
json_file = open('/kaggle/input/trainedmode/model_data.json',"r")
loaded_model = json_file.read()
json_file.close()

model = model_from_json(loaded_model)
model.load_weights('/kaggle/input/trainedmode/model_data.h5')

# **Output Dictionary**

In [ ]:
preds_dict = {
    'grapheme_root': [],
    'vowel_diacritic': [],
    'consonant_diacritic': []
}

# **Load Test Data Files And Predict Output**

In [ ]:
components = ['consonant_diacritic', 'grapheme_root', 'vowel_diacritic']
target=[] 
row_id=[] 

In [ ]:
for i in range(4):
    Y_test_img = pd.read_parquet('/kaggle/input/bengaliai-cv19/test_image_data_{}.parquet'.format(i)) 
    Y_test_img.set_index('image_id', inplace=True)

    X_test = Y_test_img.values.reshape(-1,137,236,1)
    
    preds = model.predict(X_test)

    for i, p in enumerate(preds_dict):
        preds_dict[p] = np.argmax(preds[i], axis=1)

    for k,id in enumerate(Y_test_img.index.values):  
        for i,comp in enumerate(components):
            id_sample=id+'_'+comp
            row_id.append(id_sample)
            target.append(preds_dict[comp][k])
    del Y_test_img
    del X_test


In [ ]:
sample_submission = pd.DataFrame(
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)
            #sample submission.to_csv('submission.csv',index=False)   
sample_submission.head()